In [54]:
import numpy as np
import pandas as pd 
from datasets import load_dataset
import tqdm as tqdm
import matplotlib.pyplot as plt
import plotly.express as px


In [55]:
# regression_dataset = load_dataset("ada-datadruids/regression_dataset_tmdb")
# regression_dataset_df = regression_dataset['train'].to_pandas()
books_movie_dataset = pd.read_csv("../data/final_dataset.csv")
print(books_movie_dataset.columns)
similarites = load_dataset("ada-datadruids/bookfilm_summaries_with_similarity_and_sentiment")
similarites_df = similarites['train'].to_pandas()


Index(['Unnamed: 0', 'movie_id', 'movie_cluster', 'movie_name', 'release_date',
       'runtime_x', 'language', 'country', 'genres', 'movie_year', 'index',
       'id', 'BookTitle', 'BookYear', 'Author', 'full name', 'tmdb_id',
       'id_goodreads', 'imdbid', 'runtime_y', 'revenue', 'budget', 'rb_ratio',
       'book_id', 'normalized_rating_x', 'standardized_rating_x',
       'normalized_rating_y', 'standardized_rating_y', 'length',
       'review_count'],
      dtype='object')


In [56]:
books_movie_dataset_n= pd.merge(books_movie_dataset, similarites_df, on = 'movie_id', how ='left')
books_movie_dataset_n.dropna(subset='similarity', inplace=True)
books_movie_dataset_n['similarity']

5       0.451876
6       0.559594
7       0.344548
8       0.669707
14      0.852735
          ...   
1857    0.704500
1861    0.677893
1863    0.686086
1864    0.762416
1866    0.825776
Name: similarity, Length: 837, dtype: float64

In [57]:
from utils.general_utils import adjust_for_inflation_final_dataset
#books_movie_dataset_cleaned = books_movie_dataset[books_movie_dataset['revenue'] != 0]
books_movie_dataset_cleaned = books_movie_dataset_n[books_movie_dataset_n['revenue'] != 0]
books_movie_dataset_cleaned = adjust_for_inflation_final_dataset(books_movie_dataset, books_movie_dataset_cleaned)

In [58]:
book_adaptations = books_movie_dataset_cleaned.groupby(['book_id', 'BookTitle']).size().reset_index(name='adaptation_count')

# Sort by 'adaptation_count' in descending order to see the most frequently adapted books
book_adaptations = book_adaptations.sort_values(by='adaptation_count', ascending=False)

# Display the top books with the most adaptations
print(book_adaptations[book_adaptations.adaptation_count > 1])

        book_id                              BookTitle  adaptation_count
370  35031085.0                           Frankenstein                 7
245    897831.0                           The Exorcist                 5
123    126232.0                                   Jaws                 4
65      21853.0                            First Blood                 4
151    236093.0             The Wonderful Wizard of Oz                 4
140    180617.0            The Adventures of Pinocchio                 4
60      18254.0                           Oliver Twist                 3
89      49540.0               Les Liaisons dangereuses                 3
333   3148286.0                          Anna Karenina                 3
50      14935.0                  Sense and Sensibility                 3
57      17184.0                      The Invisible Man                 3
66      23814.0                   King Solomon's Mines                 3
189    511614.0         The Hundred and One Dalmati

In [59]:
Planete_singes=books_movie_dataset_cleaned[books_movie_dataset_cleaned['BookTitle']=='La Planète des singes']
frankenstein = books_movie_dataset_cleaned[books_movie_dataset_cleaned['BookTitle']=='Frankenstein']
exorcist= books_movie_dataset_cleaned[books_movie_dataset_cleaned['BookTitle']=='The Exorcist']
peter_pan=books_movie_dataset_cleaned[books_movie_dataset_cleaned['BookTitle']=="Peter Pan, or The Boy Who Wouldn't Grow Up"]
trois_mousquetaires=books_movie_dataset_cleaned[books_movie_dataset_cleaned['BookTitle']=='Les Trois Mousquetaires']

In [60]:
books_with_multiple_films = ['La Planète des singes', 'Frankenstein',"The Exorcist","Peter Pan, or The Boy Who Wouldn't Grow Up", 'Les Trois Mousquetaires']
books_movie_dataset_cleaned['BookIdentifier'] = books_movie_dataset_cleaned['BookTitle'].where(books_movie_dataset_cleaned['BookTitle'].isin(books_with_multiple_films))
books_filtered = books_movie_dataset_cleaned[books_movie_dataset_cleaned['BookIdentifier'].notnull()]

In [61]:
fig = px.scatter(
    books_filtered,
    x='movie_year',
    y='AdjustedRevenue',
    color='BookIdentifier',
    hover_data={'BookTitle': True, 'BookYear': True, 'movie_year': True},
    title='Revenue for different Books adapted multiple Times',
    labels={'BookYear': 'Book Release Year', 'years_to_movie': 'Years to Movie Release'},
    template='plotly_white'
)

fig.update_traces(
    hovertemplate='<b>Book Title:</b> %{customdata[0]}<br>' +
                  '<b>Book Release Year:</b> %{customdata[1]}<br>' +
                  '<b>Movie Release Year:</b> %{x}<br>' +
                  '<b>Years to Movie Release:</b> %{y}<br>'
)

fig.show()


In [62]:
Planete_singes

,Unnamed: 0,movie_id,movie_cluster,movie_name_x,release_date,runtime_x,language,country,genres,movie_year,...,book_summary,film_summary,similarity,film_sentiment,film_sentiment_score,book_sentiment,book_sentiment_score,Year,CPI,AdjustedRevenue


In [63]:
frankenstein.columns

Index(['Unnamed: 0', 'movie_id', 'movie_cluster', 'movie_name_x',
       'release_date', 'runtime_x', 'language', 'country', 'genres',
       'movie_year', 'index', 'id', 'BookTitle', 'BookYear', 'Author',
       'full name', 'tmdb_id', 'id_goodreads', 'imdbid', 'runtime_y',
       'revenue', 'budget', 'rb_ratio', 'book_id', 'normalized_rating_x',
       'standardized_rating_x', 'normalized_rating_y', 'standardized_rating_y',
       'length', 'review_count', 'book_title', 'movie_name_y', 'book_summary',
       'film_summary', 'similarity', 'film_sentiment', 'film_sentiment_score',
       'book_sentiment', 'book_sentiment_score', 'Year', 'CPI',
       'AdjustedRevenue'],
      dtype='object')